In [4]:
import pandas as pd
from scipy.special import rel_entr
import numpy as np
import requests
from bs4 import BeautifulSoup
import string
import pyarrow.parquet as pq
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from datetime import datetime, timedelta

stopwords = set(stopwords.words('english'))

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')

In [5]:
from pymongo import MongoClient
import datetime

client = MongoClient('mongodb://localhost:27017/')
db = client['mydatabase']

west_virginia_collection = db['sdsc-west-virginia']
utah_collection = db['sdsc-utah']
montana_collection = db['sdsc-montana']

In [6]:
def find_keywords(text):
    tokens = word_tokenize(period_doc)
    manual_filter = ['’','”','“','—']

    filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    filtered_tokens = [token for token in filtered_tokens if token not in string.punctuation]
    filtered_tokens = [token for token in filtered_tokens if token not in manual_filter]

    tagged_tokens = pos_tag(filtered_tokens)

    desired_pos = ['NN', 'NNS', 'NNP', 'NNPS']

    keywords = [token for token, pos in tagged_tokens if pos in desired_pos]
    
    return keywords

In [17]:
west_virginia = west_virginia_collection.find()
west_virginia_filtered = []

for doc in west_virginia:
    timestamp = datetime.datetime.fromtimestamp(doc['publishdate'] / 1000)
    year = timestamp.year
    month = timestamp.month
    date = f'{month}-{year}'
    
    west_virginia_filtered.append({'news': doc['news'], 'title': doc['title'], 'publishdate': date})
    
west_virginia_df = pd.DataFrame(west_virginia_filtered)
west_virginia_df

,news,title,publishdate
0,Trump made 22 of the false claims at a campaig...,From an imaginary restraining order to a phan...,12-2019
1,"But so far, the report has remained out of the...",The Bransfield report: Post publishes secret ...,12-2019
2,War broke out between President Donald Trump a...,IT`S WAR: Trump`s feud with Steve Bannon is a...,1-2018
3,In addition to keeping the government operatin...,Stopgap Bill to End Government Shutdown Passe...,1-2018
4,Thumbs up in West Virgina: President Donald Tr...,"In West Virginia, Trump Shuns One of the GOP’...",4-2018
...,...,...,...
105,WASHINGTON — The Drug Enforcement Administrati...,Watchdog finds DEA was ‘slow to respond’ to o...,10-2019
106,Trees have been cut for the Atlantic Coast Pip...,U.S. Supreme Court to hear case of gas pipeli...,10-2019
107,(Reuters) - Several U.S. states that have been...,Several states wary of $48 billion opioid set...,10-2019
108,(Reuters) - Several U.S. states that have been...,Several states wary of $48 billion opioid set...,10-2019


In [21]:
utah = utah_collection.find()
utah_filtered = []

for doc in utah:
    timestamp = datetime.datetime.fromtimestamp(doc['publishdate'] / 1000)
    year = timestamp.year
    month = timestamp.month
    date = f'{month}-{year}'
    
    utah_filtered.append({'news': doc['news'], 'title': doc['title'], 'publishdate': date})
    
utah_df = pd.DataFrame(utah_filtered)
utah_df

,news,title,publishdate
0,"The answer is obvious. Change is coming, and i...",Don’t let NCAA suits decide college sports’ f...,12-2019
1,(Pep Montserrat for The Washington Post)\n\nWi...,The Daily 202: Alexander Hamilton has been ca...,12-2019
2,President Trump is likely to enter the history...,Trump impeachment: What happens next?,12-2019
3,President Trump entered the history books in a...,What`s next in the impeachment process?,12-2019
4,President Trump is likely to enter the history...,What’s next in Trump impeachment,12-2019
...,...,...,...
913,There are all sorts of ideas floating around a...,How far can the House go to stop a sham trial?,12-2019
914,These Democrats characterized their choices as...,Centrist Democrats line up behind impeachment,12-2019
915,These Democrats characterized their choices as...,Centrist Democrats line up behind impeachment,12-2019
916,Washington (CNN) It may be a no-brainer for Se...,Democrats try to make Republicans pay the pri...,12-2019


In [20]:
montana = montana_collection.find()
montana_filtered = []

for doc in montana:
    timestamp = datetime.datetime.fromtimestamp(doc['publishdate'] / 1000)
    year = timestamp.year
    month = timestamp.month
    date = f'{month}-{year}'
    
    montana_filtered.append({'news': doc['news'], 'title': doc['title'], 'publishdate': date})
    
montana_df = pd.DataFrame(montana_filtered)
montana_df

,news,title,publishdate
0,"“It’s really about dignity, because we’ve been...",A big moment finally comes for the Little She...,12-2019
1,Trump made 22 of the false claims at a campaig...,From an imaginary restraining order to a phan...,12-2019
2,(CNN) Republicans and Democrats will sit down ...,DACA negotiations hit a snag ahead of White H...,1-2018
3,"But the House measure, put forth by a group th...",House Republicans’ Hard-Line Immigration Stan...,1-2018
4,The White House announced it will release a bi...,Democrats are starting to acknowledge the ine...,1-2018
...,...,...,...
152,There are still more than a dozen candidates j...,Opinion | The Presidency Is Not Enough,11-2019
153,There are still more than a dozen candidates j...,Opinion | The Presidency Is Not Enough,11-2019
154,There are still more than a dozen candidates j...,Opinion | The Presidency Is Not Enough,11-2019
155,Russell Dotson served 22 years with the U.S. N...,The Navy told him he served long enough to ge...,11-2019


In [22]:
utah_df.to_csv('utah.csv', index = False)
montana_df.to_csv('montana.csv', index = False)
west_virginia_df.to_csv('west_virginia.csv', index = False)

In [ ]:
start_date = pd.to_datetime('2019-12-18')
end_date = pd.to_datetime('2020-12-19')

In [8]:
period = df[(df['collectiondate'] >= start_date) & (df['collectiondate'] <= end_date)][:50]
period_doc = ' '.join(testdf['news'].astype(str))
period_keywords = find_keywords(period_doc)
top_period_keywords = {x[0] : x[1]/len(period_doc) for x in Counter(period_keywords).most_common(5)}

NameError: name 'df' is not defined

In [ ]:
current_date = start_date
out = []

while current_date <= end_date:
    day_df = df[df['collectiondate'] == pd.to_datetime(current_date)]
    day_doc = ' '.join(day_df['news'].astype(str))
    day_keywords = find_keywords(day_doc)
    day_counter = Counter(day_keywords)
    out.append({x : day_counter[x]/len(day_doc) for x in top_period_keywords})
    
    current_date += timedelta(days=1)

In [ ]:
#KL divergence date vs previous date
for i in range(1,len(out)):
    print(sum(rel_entr(list(out[i-1].values()),list(out[i].values()))))

In [70]:
#KL divergence date vs total
for dists in out:
    print(sum(rel_entr(list(period_keywords.values()),list(dists.values()))))

0.0
0.0007404775326340417
